In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/pope/popular/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,label,question_id,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a traffic light in the image?\nAnswe...,[tensor(1)],[tensor(2271)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there an umbrella in the image?\nAnswer th...,[tensor(1)],[tensor(5707)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a car in the image?\nAnswer the ques...,[tensor(0)],[tensor(3660)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a chair in the image?\nAnswer the qu...,[tensor(0)],[tensor(6478)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a fork in the image?\nAnswer the que...,[tensor(1)],[tensor(10623)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None


In [5]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [6]:
embeddings=np.concatenate(embeddings,axis=0)

In [7]:
embeddings.shape

(10000, 29, 3584)

In [8]:
gt_label=df['label'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [9]:
gt_label.shape

(10000,)

In [10]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [11]:
is_correct=[]
for i,x in enumerate(resps):
    if x.lower()=='no' and gt_label[i]==0:
        is_correct.append(1)
    elif x.lower()=='yes' and gt_label[i]==1:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [12]:
is_correct.shape

(10000,)

In [13]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly hall ratio {sum(is_correct==0)/len(is_correct)*100}%')

There are 10000 responses. 8944 answer correctly and 1056 answer wrongly hall ratio 10.56%


In [14]:
n_samples=len(is_correct)

indices = np.where(is_correct == 1)[0]
indices=np.array(indices)

n_correct=len(indices)
print(n_correct)
hall_ratio=0.6

x=(n_correct-n_samples*(1-hall_ratio))/hall_ratio
print(x)
np.random.shuffle(indices)
remove_indices=indices[:int(x)]
is_correct=np.delete(is_correct, remove_indices)

embeddings=np.delete(embeddings, remove_indices,axis=0)

8944
8240.0


In [15]:
embeddings.shape

(1760, 29, 3584)

In [16]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly hall ratio {sum(is_correct==0)/len(is_correct)*100}%')

There are 1760 responses. 704 answer correctly and 1056 answer wrongly hall ratio 60.0%


In [17]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 100
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:1308
validset length:100
testset length:352
trainset postive: 521 negative: 787
valset postive: 43 negative: 57
testset postive: 140 negative: 212


In [18]:
all_embeddings = embeddings
all_embeddings.shape

(1760, 29, 3584)

In [19]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [20]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel,KernelPCA
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score
from ml_utils.metrics import auc_pr

Grid Search for Best 'n_components' & 'layer'

In [21]:
M=3584
def evaluator(n_components,i_layer,X,y):
    discriminator=KernelPCA(embedding_train[:,i_layer,:],n_components,M,method='CoP',gamma=1)
    scores=discriminator.get_score(X[:,i_layer,:])
    
    return roc_auc_score(y,scores)
    #split=discriminator.get_best_split(scores,y)
    #y_pred=scores>split
    #return accuracy_score(y,y_pred)
    #return balanced_accuracy_score(y,y_pred)
    #
    #split=discriminator.get_best_split(y)
    #y_preds=(scores>split)
    #return accuracy_score(y,y_preds)

In [22]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,3),
    'layer':range(10,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras,all_results=grid_search.search(return_results=True)

Grid Searching for best n_components,layer


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:1,layer:10 Score: 0.5201958384332926
found best n_components:1,layer:16 Score: 0.70093839249286
found best n_components:1,layer:17 Score: 0.8363933088535292
found best n_components:1,layer:18 Score: 0.8486332109343125


  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:2,layer:19 Score: 0.857609139126887
found best n_components:2,layer:21 Score: 0.8759689922480619


In [23]:
#np.save('./pope_layers.npy',all_results,allow_pickle=True)

In [24]:
best_paras

{'n_components': 2, 'layer': 21, 'best_score': np.float64(0.8759689922480619)}

In [25]:
best_layer=best_paras['layer']
#best_layer=24

View Results on Training set and Test set

In [26]:
discriminator=KernelPCA(embedding_train[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [27]:
scores=discriminator.get_score(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.7660550458715596 AUROC:0.8330256300194866


In [28]:
discriminator=KernelPCA(embedding_test[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.6051136363636364 AUROC:0.8446428571428571


Train Linear Prob to Get Better Results

In [29]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [30]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [31]:
torch.save(clf.state_dict(), './pope_clf.pt')

In [32]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores_test)}  b-acc:{balanced_accuracy_score(y_test,pred_test)} auc-pr:{auc_pr(y_test,scores_test)}')

test set acc: 0.7784090909090909 f1:0.75 auroc:0.838527628032345  b-acc:0.7881401617250674 auc-pr:0.794117979002636
